# Ahmet Furkan Ün
# EM1SNO

## Exercise 1

In [159]:
import math

def newtons_method(f, df, x0, precision):
    xk = x0
    iterations = 0
    while abs(f(xk)) > precision:
        if df(xk) != 0:
            xk = xk - f(xk) / df(xk)
            iterations += 1
        else:
            print(f"Iteration {iterations}. Derivative is 0. No solution!")
            break
    return xk, iterations

def secant_method(f, x0, x1, precision):
    xk_1 = x0
    xk = x1
    iterations = 0
    while abs(f(xk)) > precision:
        prev_xk = xk
        xk = xk - f(xk) * (xk - xk_1) / (f(xk) - f(xk_1))
        xk_1 = prev_xk
        iterations += 1
    return xk, iterations

def bisection_method(f, a, b, precision):
    iterations = 0
    c = (a + b) / 2
    
    if f(a) * f(b) > 0:
        print("The function has no root in the given interval. Make sure f(a) * f(b) < 0")
        
    elif f(a) * f(b) < 0:
        c = (a + b) / 2
        while abs(f(c)) > precision:
            if f(c) == 0:
                return c, iterations
            elif f(c) * f(b) > 0:
                b = c
            elif f(c) * f(b) < 0:
                a = c

            c = (a + b) / 2
            iterations += 1
    else:
        if f(a) == 0:
            return a, iterations
        else:
            return b, iterations
    return c, iterations

## Exercise 2 

### (a) 


#### Show that there is a solution in [1, 2], 

$$
f(1) = 1^3 + 2 \cdot 1^2 + 10 \cdot 1 - 20 = -7
$$
$$
f(2) = 2^3 + 2 \cdot 2^2 + 10 \cdot 2 - 20 = 16
$$

Since $ f(1) = -7 $ and $ f(2) = 16 $, by the **Intermediate Value Theorem**, there is at least one root in the interval $[1, 2]$ because the function changes signs.


#### Show there is only one real solution on $ \mathbb{R} $.
The derivative of $ f(x) $ is:

$$
f'(x) = 3x^2 + 4x + 10
$$
$$
d = b^2 - 4ac = 4^2 - 4\cdot 3 \cdot 10 = -104 < 0
$$

Since the discriminant of $ f'(x)$ is negative, there are no real roots for the derivative and $ f'(x) > 0 $ for all $ x \in \mathbb{R} $. Therefore, $ f(x) $ is a strictly increasing function on $ \mathbb{R} $, which means there is only one real root.


In [2]:
def f(x):
    return x**3 + 2*x**2 + 10*x - 20

def df(x):
    return 3*x**2 + 4*x + 10

precision = 1e-4

### (b) 

In [3]:
root_newton, iters_newton = newtons_method(f, df, 1, precision)
print(f"Newton's Method: Root = {root_newton}, Iterations = {iters_newton}")

Newton's Method: Root = 1.3688081886175318, Iterations = 3


### (c) 

In [4]:
root_secant, iters_secant = secant_method(f, 1, 2, precision)
print(f"Secant Method: Root = {root_secant}, Iterations = {iters_secant}")

Secant Method: Root = 1.3688074597219246, Iterations = 4


### (d) 

In [160]:
root_bisection, iters_bisection = bisection_method(f, 1, 2, precision)
print(f"Bisection Method: Root = {root_bisection}, Iterations = {iters_bisection}")

Bisection Method: Root = 1.368804931640625, Iterations = 14


## Exercise 3

In [35]:
def f_tanh(x):
    return math.tanh(x)

def df_tanh(x):
    return 1 - f_tanh(x)**2  # First derivative

def d2f_tanh(x):
    return -2 * f_tanh(x) * (1 - f_tanh(x)**2)   # Second derivative

def d3f_tanh(x):
    return 2 * (1 - f_tanh(x)**2) * (3 * f_tanh(x)**2 - 1)   # Third derivative

### Newton's Method
Newton’s method works by iteratively improving an estimate $ x_k $ using the formula:
$$
x_{k+1} = x_k - \frac{f(x_k)}{f'(x_k)}
$$
However, the equation $ f(x) = \tanh(x) = 0 $ presents challenges because $ \tanh(x) $ is nearly flat at large negative and oistive values, causing its derivative $ f'(x) $ to approach zero. This can lead to division by zero or cause very small updates, slowing convergence. Hence, an adaptive approach is needed to handle cases where the derivative is too small or when we encounter flat regions.

In the updated algorithm, when the derivative is large enough, we use standard Newton’s method. If $ f'(x_k) $ is close to zero, we switch to alternative approaches using higher-order derivatives (2nd and 3rd), namely Method II in paper called "A Method to Avoid the Division-by-Zero, Near-Zero, Divergence, and Oscillation in Newton’s Method". This avoids division by zero or slow convergence. If the second derivative $ f''(x) $ is small and the third derivative is non-zero, we apply a correction based on the cube root to escape slow convergence or oscillations. If all else fails, we introduce a small $ \epsilon $ to the denominator to avoid division by zero, ensuring the method progresses.

https://www.researchgate.net/publication/358857049_A_Method_to_Avoid_the_Division-by-Zero_or_Near-Zero_in_Newton-Raphson_Method

In [138]:
def updated_newtons_method(f, df, d2f, d3f, x0, precision, epsilon=0.05, max_iter=1000):
    xk = x0
    iterations = 0

    while abs(f(xk)) > precision and iterations < max_iter:
        if abs(df(xk)) > epsilon:
            # Perform Newton's method when derivative is sufficiently large
            xk = xk - f(xk) / df(xk)
        elif abs(d2f(xk)) > 1e-10:
            x1 = xk + 2*df(xk)/d2f(xk)
            if abs(d3f(x1)) > 1e-10:
                temp = 6*f(x1)/d3f(x1)
                real_cube_root = temp ** (1/3) if temp >= 0 else -(-temp) ** (1/3)
                xk = x1 - real_cube_root
            elif abs(d2f(x1)) > 1e-10:
                xk = x1 - math.sqrt(-2*f(x1)/d2f(x1))
        else:
            xk = xk - f(xk) / (df(xk)+epsilon) # if nothing works, add small amount to denominator to get rid of divison by near zero
            
        iterations += 1
 
    if abs(f(xk)) <= precision:
        return xk, iterations
    else:
        print("Maximum iterations reached without convergence.")
        return None, iterations

root_newton, iters_newton = updated_newtons_method(f_tanh, df_tanh,d2f_tanh, d3f_tanh, -5, precision)
print(f"Newton's Method: Root = {root_newton}, Iterations = {iters_newton}")

Newton's Method: Root = 4.620871738934829e-06, Iterations = 6


### Secant Method
Secant Method also have similar problem with Newton's Method since $\tanh(x) $ is nearly flat at large negative and poistive values, causing its the denominator $ f(x_k) − f(x_{k−1}) $ to approach zero.
Using just two points is not sufficient in this case. Instead of using two points to approximate the derivative, we use three points to create a quadratic function that better approximates the behavior of $ f(x) $. This method is known as Inverse Quadratic Interpolation. Using three points allows us to approximate the root more effectively in some cases, improving convergence speed compared to the standard secant method.

In [191]:
def updated_secant_method(f, x0, x1, precision, max_iter=1000):
    #Instead of using 2 points to interpolate, we can use 3 points. This method is called Inverse Quadratic Interpolation
    x2 = x1 + (x1-x0) # This method needs 3 points. 3rd point is choosen such that 3 points have same distance
    iterations = 0
    while iterations < max_iter:
        if abs(f(x0)) < precision:
            return x0, iterations
        if abs(f(x1)) < precision:
            return x1, iterations
        if abs(f(x2)) < precision:
            return x2, iterations
        
        denominator = (f(x0) * (1 / (x1 - x2)) + 
                       f(x1) * (1 / (x2 - x0)) + 
                       f(x2) * (1 / (x0 - x1)))
        
        # Avoid division by zero
        if abs(denominator) < 1e-10:
            print("Denominator too small, exiting.")
            break
        
        x3 = (f(x0) * (x1 - x2) + 
              f(x1) * (x2 - x0) + 
              f(x2) * (x0 - x1)) / denominator

        # Update points for next iteration
        x0, x1, x2 = x1, x2, x3
        iterations += 1
    return x2, iterations
    



root_secant, iters_secant = updated_secant_method(f_tanh, -5, -4, precision)
print(f"Secant Method: Root = {root_secant}, Iterations = {iters_secant}")


Secant Method: Root = -2.439744057834881e-06, Iterations = 5


### Bisection Method
The bisection method requires that the function has opposite signs at the endpoints $ a $ and $ b $ (i.e., $ f(a) \cdot f(b) < 0 $), which guarantees that a root exists within the interval. In our case both endpoints are positive ($ f(5) > 0$ and $ f(10) > 0$). In the updated algorithm, before starting the root-finding process, we check whether the interval $[a, b]$ contains a root by verifying that $ f(a) \cdot f(b) < 0 $. If it doesn’t (i.e., both values have the same sign), we extend the interval outward by adjusting $ a $ and $ b $ symmetrically. The step size starts small and doubles with each iteration, allowing us to explore a wider range if needed. Once we confirm that the interval contains a root, we start the bisection process

In [158]:
def updated_bisection_method(f, a, b, precision, step_size=0.5, max_update=100):
    iterations = 0
    update_count = 0
    c = (a + b) / 2
    
    if f(a) * f(b) > 0:
        print("Your interval do not contain the root. Updating the interval...")
        while f(a) * f(b) > 0 and update_count < max_update:
            a -= step_size
            b += step_size
            step_size *= 2
            update_count += 1
            
        if f(a) * f(b) > 0:
            print("The function has no root near the provided interval. To search wider, increase 'max_update' or check your interval")
            
        elif f(a) * f(b) < 0:
            print(f"Updated interval: [{a}, {b}]")   
        
    elif f(a) * f(b) < 0:
        c = (a + b) / 2
        while abs(f(c)) > precision:
            if f(c) == 0:
                return c, iterations
            elif f(c) * f(b) > 0:
                b = c
            elif f(c) * f(b) < 0:
                a = c

            c = (a + b) / 2
            iterations += 1
    else:
        if f(a) == 0:
            return a, iterations
        else:
            return b, iterations
    return c, iterations

root_bisection_tanh, iters_bisection_tanh = updated_bisection_method(f_tanh, 5, 10, precision)
print(f"Bisection Method: Root = {root_bisection_tanh}, Iterations = {iters_bisection_tanh}")


Your interval do not contain the root. Updating the interval...
Updated interval: [-2.5, 17.5]
Bisection Method: Root = 7.5, Iterations = 0


## Exercise 4

$$
g(x) = \frac{20}{x^2 + 2x + 10}
$$

To apply the Contraction Mapping Theorem, we need to show that:

1. **$ g(x) $ maps the interval $[1, 2]$ into itself.**
   

2. **$ g(x) $ is a contraction on $[1, 2]$.**


### 1. Checking if $ g(x) $ maps the interval $ [1, 2] $ into itself
We need to confirm that for all $ x \in [1, 2] $, $ 1 \leq g(x) \leq 2 $. 
As g is a continuous function and [1, 2] is a bounded and closed interval, g has a minimum or maximum values which either occur at endpoints or where $g'(x) = 0$

We have:
$$
g(x) = \frac{20}{x^2 + 2x + 10}
$$
Using the quotient rule:
$$
g'(x) = -\frac{20 \cdot (2x + 2)}{(x^2 + 2x + 10)^2}
$$
Simplifying:
$$
g'(x) = -\frac{40(x + 1)}{(x^2 + 2x + 10)^2}
$$

which is defined for all  $ x \in [1, 2] $ and there is no $ x \in [1, 2] $ such that $g'(x) =0$. Therefore minimum and maximum values are endpoints of the interval.

$$
g(1) = \frac{20}{1^2 + 2(1) + 10} = \frac{20}{13} \approx 1.538
$$
$$
g(2) = \frac{20}{2^2 + 2(2) + 10} = \frac{20}{18} \approx 1.111
$$

Thus, for all $ x \in [1, 2] $, the value of $ g(x) $ stays between $ g(1) \approx 1.538 $ and $ g(2) \approx 1.111 $. This confirms that $ g(x) \in [1, 2] $ for all $ x \in [1, 2] $, and thus $ g(x) $ maps the interval $ [1, 2] $ into itself.

### 2. Checking if $ g(x) $ is a contraction on the interval $ [1, 2] 

We need to show that $ g(x) $ is a contraction, meaning that there exists a constant $ L \in [0, 1) $ such that:

$$
|g(x_1) - g(x_2)| \leq L |x_1 - x_2| \quad \text{for all} \ x_1, x_2 \in [1, 2]
$$

To verify this, we compute the derivative $ g'(x) $ and show that $ |g'(x)| \leq L $ for some $ L < 1 $ on $ [1, 2] $.

As previously calculated:
$$
g'(x) = -\frac{40(x + 1)}{(x^2 + 2x + 10)^2}
$$

Now, we need to check the absolute value of $ g'(x) $ in the interval $ [1, 2] $.

#### Evaluating $ |g'(x)| $ in $ [1, 2] $

We know that:
$ 1 \leq x \leq 2$

Therefore:

$ 80 \leq 40(x + 1) \leq 120$

And

$ 169 \leq (x^2 + 2x + 10)^2 \leq 324$



Therefore $ |g'(x)| < 1 $ for all $ x \in [1, 2] $ and $ g(x) $ is a contraction.



Since $ g(x) $ is a contraction and maps the interval $ [1, 2] $ into itself, the Contraction Mapping Theorem guarantees that the simple iteration:

$$
x_{k+1} = g(x_k)
$$

will converge to a unique fixed point $ \xi \in [1, 2] $, which is the solution to $ f(x) = 0 $.


In [33]:
def g(x):
    return 20 / (x**2 + 2*x + 10)

def simple_iteration(g, x0, precision):
    xk = x0
    iterations = 0
    while abs(g(xk) - xk) > precision:
        xk = g(xk)
        iterations += 1
    return xk, iterations

x0 = 1.5
x1 = g(x0)
L = 80/169
for dec in range(2, 17, 2):
    precision = 10**(-dec)
    eps, iteration = simple_iteration(g, x0, precision)
    i_theoretical = math.floor((math.log(abs(x0-x1)) - math.log(precision*(1-L)))/(math.log(1/L))) + 1
    print(f"Precision: 1e-{dec:<2}  Root: {round(eps, dec):<18}  # of Iterations: {iteration:<4}  Theoretical Criterion: {i_theoretical:<4}")

Precision: 1e-2   Root: 1.37                # of Iterations: 4     Theoretical Criterion: 5   
Precision: 1e-4   Root: 1.3688              # of Iterations: 10    Theoretical Criterion: 11  
Precision: 1e-6   Root: 1.368807            # of Iterations: 15    Theoretical Criterion: 18  
Precision: 1e-8   Root: 1.3688081           # of Iterations: 21    Theoretical Criterion: 24  
Precision: 1e-10  Root: 1.3688081078        # of Iterations: 27    Theoretical Criterion: 30  
Precision: 1e-12  Root: 1.368808107822      # of Iterations: 32    Theoretical Criterion: 36  
Precision: 1e-14  Root: 1.36880810782138    # of Iterations: 38    Theoretical Criterion: 42  
Precision: 1e-16  Root: 1.3688081078213725  # of Iterations: 43    Theoretical Criterion: 48  


The theoretical criterion consistently predicts a slightly larger number of iterations compared to the actual number of iterations performed by my implementation. This shows that the theoretical criterion is indeed pessimistic, though not excessively so.